<a href="https://colab.research.google.com/github/ShinyaKatoh/Trial_Polarity/blob/main/model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ShinyaKatoh/Trial_Polarity
!pip install einops
!pip install torchinfo

Cloning into 'Trial_Polarity'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16 (from 1)
Receiving objects: 100% (16/16), 46.01 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
# Google driveにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/Trial_Polarity')

In [5]:
import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import DataLoader, Dataset

import torch.nn.functional as F
import torch.nn.init as init

import os
import glob
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

import util

In [6]:
# モデル構造の定義
#
# model1 : Neural Network
#
# model2 : Convolutional Neural Network
#
# model3 : Convolutional Neural Network + Regularization
#

import model1

model = model1.Model()

# モデル保存のディレクトリの指定
save_dir = '/content/drive/MyDrive/model1'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# パラメータ設定
#-------------------------------------------------------------------------------------------------------------------------------

# バッチサイズ
batch_size = 128

#エポック数
num_epochs = 100

#-------------------------------------------------------------------------------------------------------------------------------
# デバイスの割り当て
#-------------------------------------------------------------------------------------------------------------------------------

device = torch.device(("cuda" if torch.cuda.is_available() else "cpu"))
print(device)

#-------------------------------------------------------------------------------------------------------------------------------
# データの読み込み
#-------------------------------------------------------------------------------------------------------------------------------

train_dataset = util.MyDataset('/content/Trial_Polarity/data/train_data.pt', '/content/Trial_Polarity/data/train_label_for_CLA.pt')
train_loader = util.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

valid_dataset = util.MyDataset('/content/Trial_Polarity/data/valid_data.pt', '/content/Trial_Polarity/data/valid_label_for_CLA.pt')
valid_loader = util.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

#-------------------------------------------------------------------------------------------------------------------------------
# 学習の実行
#-------------------------------------------------------------------------------------------------------------------------------

# モデル構造の転送
model.to(device)

# パラメータの初期化
model.apply(util.init_weights)

# モデル構造の表示
print(summary(model, input_size=(1,1,128)))

# 最適化アリゴリズムの設定
optimizer = torch.optim.Adam(model.parameters())

# 学習の実行
history = np.zeros((0,3))
history = util.fit(model, optimizer, num_epochs, train_loader, valid_loader, device, history, save_dir)

# 学習過程の保存
np.save(save_dir+'/history', history)

cpu


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 3]                    --
├─Linear: 1-1                            [1, 128]                  16,512
├─ReLU: 1-2                              [1, 128]                  --
├─Linear: 1-3                            [1, 128]                  16,512
├─ReLU: 1-4                              [1, 128]                  --
├─Linear: 1-5                            [1, 128]                  16,512
├─ReLU: 1-6                              [1, 128]                  --
├─Linear: 1-7                            [1, 64]                   8,256
├─ReLU: 1-8                              [1, 64]                   --
├─Linear: 1-9                            [1, 32]                   2,080
├─ReLU: 1-10                             [1, 32]                   --
├─Linear: 1-11                           [1, 3]                    99
├─Softmax: 1-12                          [1, 3]                    

Epoch Train 1/100:   0%|          | 0/793 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch Valid 1/100: 100%|██████████| 100/100 [00:01<00:00, 83.37it/s]


Epoch [1/100], loss: 28.53585 acc: 0.62355 val_loss: 25.08379 val_acc: 0.67589


...

Epoch Valid 31/100: 100%|██████████| 100/100 [00:00<00:00, 167.40it/s]


Epoch [31/100], loss: 3.64026 acc: 0.96855 val_loss: 22.02000 val_acc: 0.87677


Epoch Valid 32/100: 100%|██████████| 100/100 [00:00<00:00, 136.80it/s]


Epoch [32/100], loss: 3.62133 acc: 0.96872 val_loss: 21.19200 val_acc: 0.87386


Epoch Train 33/100:  26%|██▌       | 204/793 [00:02<00:07, 82.28it/s]